In [ ]:
from pymatgen.io.openmm.generators import OpenMMSolutionGen
from pymatgen.io.openmm.sets import OpenMMSet
from pymatgen.io.openmm.simulations import equilibrate_pressure, anneal

from openmm.app import StateDataReporter, DCDReporter, PDBReporter
from sys import stdout


## Generators


In [ ]:
# input sets contain all of the files needed to run a simulation!
# it just writes and reads input files

# generator makes input sets!
# specified settings, but no molecules!


In [ ]:
openmm_generator = OpenMMSolutionGen(
    force_field = "Sage",
    temperature = 298,
    step_size = 0.001,
    partial_charge_method = "mmff94",
    packmol_timeout = 500 )


## Input Set

In [ ]:
trimer_smile = "MOLECULE SMILES"
molecules = {"O": 3640,"[Cl-]":560, "[OH3+]": 360, trimer_smile: 50}

input_set = openmm_generator.get_input_set(molecules, density = 0.8)


In [ ]:
input_set.write_input(f"input1")

In [ ]:
input_set = OpenMMSet.from_directory("input1")

## Simulation

In [ ]:
simulation = input_set.get_simulation()

In [ ]:
!mkdir output1

pdb_reporter = PDBReporter("output1/topology.pdb",1)
pdb_reporter.report(simulation, simulation.context.getState(getPositions=True))

n_timesteps = 1000
simulation.reporters.append(StateDataReporter("output1/state.txt", n_timesteps, step=True, potentialEnergy=True, temperature=True, volume=True, density=True))
simulation.reporters.append(DCDReporter(f"output1/trajectory.dcd", n_timesteps))


In [ ]:
simulation.minimizeEnergy()
equilibrate_pressure(simulation, 10000)
anneal(simulation, 400, [1000, 1000, 1000])  
simulation.step(50000)